## Clasificación Multiclase
Tenemos **tres estrategias** para predecir a cuál de las especies de iris pertenece un ejemplo, aplicamos una a medias (más bien a tercias) en el capítulo anterior, veamos cuál es.

Zigzag Machine learning es una colección de Jupyter Notebooks que pretende enseñar las bases del Machine Learning.
Sigue un enfoque *top-down*, de arriba hacia abajo, por lo general y *bottom-up*, de abajo hacia arriba, cuando es necesario (o interesante). Por estas subidas y bajadas tenemos un zigzag 😄

# Requisitos
Puedes utilizar [Anaconda](https://www.anaconda.com/distribution/) que tiene todo lo necesario y algunos extras.  
O [Miniconda](https://docs.conda.io/en/latest/miniconda.html) para instalar sólo lo necesario.

Cuando lo tengas configurado clona el repositorio, y dentro de él ejecuta:

    conda env create -f env.yml
    conda activate zzml
    
Finalmente abre Jupyter Lab

    $ conda activate zzml

O el clásico Jupyter Notebook

    $ conda activate zzml

### One vs the Rest
La idea es etiquetar **una** clase como **positiva** y **todas las demás** como **negativas**, es **una contra el resto**. En el capítulo anterior distinguimos entre **setosa** y **no-setosa**, esto lo hacemos para todas las clases, entrenando un clasificador **distinto** cada caso.

Esto no es muy complicado de implementar, pero por suerte `LogisticRegression` ya cuenta con la opción de usar dicha estrategia, estableciento su parámetro `multi_class` en `'ovr'`.

Este modelo espera que `y` sea un array, **no es necesario** que realicemos el one-hot encoding.

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np

np.random.seed(42)

df = pd.read_csv("datasets/Iris.csv")
y = df["Species"]

# no necesitamos estas columnas
# guardamos las demás en X
X = df.drop(columns=['Id', 'Species'])

model_log = LogisticRegression(multi_class="ovr")
model_log.fit(X, y)

# realicemos algunas predicciones
X_sample = X.sample(5)
model_log.predict(X_sample)

También tenemos la implementación en `OneVsRestClassifier` de `sklearn.multiclass`, este recibe una **instancia del modelo** que usaremos, por supuesto, debe ser uno de **clasificación**.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

model_ovr = OneVsRestClassifier( LogisticRegression() )
model_ovr.fit(X, y)
model_ovr.predict(X_sample)

### One vs One
Entrenamos clasificadores por pares, **uno contra uno** distinguimos setosa de versicolor, setosa de virginica, versicolor de virginica, y así hasta entrenar todas las combinaciones, siendo $K (K-1) / 2 $ en total, donde $K$ es el número de clases.

La ventaja es que cada clasificador se entrena **sólo con los datos que contienen las clases a distinguir**, algunos modelos **no escalan bien con el tamaño del dataset**, es decir, rinden mal con un dataset más grande.  
En estos casos es más rápido entrenar **muchos** clasificadores en **pequeños** datasets que **pocos** en datasets más **grandes**. 

Usaremos `OneVsOneClassifier` también de `sklearn.multiclass` pues `LogisticRegression` no la soporta.

In [ ]:
from sklearn.multiclass import OneVsOneClassifier

model_ovo = OneVsOneClassifier(LogisticRegression())
model_ovo.fit(X, y)
model_ovo.predict(___)

### Regresión logística multinomial
Es una **generalización** de la regresión logística, que cubre **múltiples clases** directamente, eliminando la necesidad de entrenar varios clasificadores binarios.

El parámetro `multiclass` que vimos antes acepta los parámetros `'auto'`, `'ovr'` y `'multinomial'`, **auto** es la opción por defecto, y esta selecciona **ovr** si `y` es **binario**.

Como no es nuestro caso, seleccionará **multinomial**, por lo que sería opcional especificarlo.

In [ ]:
model_multi = LogisticRegression(multi_class='multinomial')
model_multi.fit(X, y)
model_multi.predict(X_sample)

Esto también es conocido como **Regresión Softmax**. Dado un ejemplo $x$, esta calcula primero un puntaje $s_k(x)$ para cada clase $k$

$s_k(x) = x^T\theta^{(k)}$

Nota que **cada clase** tiene su propio **vector** $\theta^{(k)}$. Cada uno se guarda en una fila de la matríz $\theta$ que está repartida en los atributos `coef_` e `intercept_` de nuestro modelo. Como vimos al implementar el [descenso del gradiente](5_descenso_del_gradiente.ipynb) $\theta$ contiene los coeficientes y el término independiente en una sola matriz.

In [ ]:
 model_multi.coef_

Una columna por cada feature, una fila por cada clase 😉

Luego se aplica la función **softmax** para calcular las probabilidades $\hat{p}_k$ de que el ejemplo $x$ **pertenezca a una clase $k$** calculando una especie de media de las exponenciales, esto es $e$ elevado a los puntajes que antes calculamos.

$\hat{p}_k = \dfrac{\exp s_k(x)}{\sum^K_{j=1}\exp s_j(x)}$

Donde $K$ es el número total de clases

In [ ]:
len(model_multi.classes_)

Al final, se predice la clase con la **probabilidad más alta**, podemos usar `np.argmax` sobre un **vector** que contenga las **probabilidades**, esto nos dará el **índice** donde se encuentra el número más alto.

Primero calculemos $s(x)$ como un vector que contega los **puntajes de cada clase**, por lo que tendrá 3 elementos y será el resultado de una multiplicación de matrices, y no olvidemos sumar los términos independientes 😉

In [ ]:
# Operamos con una matriz de numpy
# y guardamos la primera fila
X_sample1 = X_sample.to_numpy()[0]

s = X_sample1 @ model_multi.coef_.T + model.intercept_
s

Luego $p$ como otro vector de tres elementos que contiene las **probabilidades** de que nuestro ejemplo **pertenezca a alguna de las clases** 0, 1 o 2, setosa, versicolor o virginica, respectivamente 😄

In [ ]:
p = np.exp(s) / sum(np.exp(s))
p

Finalmente obtenemos a qué clase pertenece nuestro ejemplo 😃

In [ ]:
# Clase de nuestro primer ejemplo
pred = np.argmax(p)
pred

Es **versicolor**, como nuestros anteriores modelos lo predijeron 😄

Ahora intenta predecir **todo** `X_sample`, al final `pred` debería ser un **array de 5 elementos** con cada una de las clases predichas 😉  
Además, `s` y `p` deberían ser matrices de 3x5 

Cada estrategia tiene su momento: **OvR** si el modelo es **estrictamente binario**, **OvO**  el modelo **no escala** bien con la **cantidad de datos** y la regresión **softmax** es como una regresión logística 😄 (generalizada)

Las primeras dos nos descubren que existen **más modelos** de clasificación, pero el próximo que veremos usa softmax 😂 y no por ello es menos interesante, es más, te aseguro que su nombre te despierta interés 🧠:

Te presento a las [redes neuronales artificiales](9_redes_neuronales.ipynb) 